In [1]:

import dotenv
import os
import requests

In [2]:
dotenv.load_dotenv()
API_KEY = os.getenv("API_KEY")

In [3]:
BASE_URL = 'https://api.pokemontcg.io/v2/'

In [ ]:
card_endpoints = {
    'card_id': 'cards/<id>',
    'sets': 'sets',
    'types': 'types',
    'subtypes': 'subtypes',
    'supertypes': 'supertypes'
}

In [21]:
class RestClient():
    api_key = None

    @classmethod
    def configure(cls, api_key):
        cls.api_key = api_key

    @classmethod
    def get(cls, url: str, params: dict={}) -> dict or None:
        """Invoke an HTTP GET request on a url
        
        Args:
            url (string): URL endpoint to request
            params (dict): Dictionary of url parameters 
        Returns:
            dict: JSON response as a dictionary
        """
        request_url = url
        
        headers = { 'User-Agent': 'Mozilla/5.0' }
        api_key = cls.api_key if cls.api_key is not None else os.getenv('POKEMONTCG_API_KEY')
        if api_key:
            headers['X-Api-Key'] = api_key

        response = requests.get(request_url, params=params, headers=headers)
        if response.status_code != 200:
            return None
        return response.json()

In [22]:
class QueryBuilder():
    def __init__(self, type):
        self.params = {}
        self.type = type
    
    def find(self, id: str) -> dict or None:
        """Get a resource by its id
        
        Args:
            id (string): Resource id    
        Returns:
            dict: Resource dictionary
        """
        url = f"{BASE_URL}/{self.type.RESOURCE}/{id}"
        response = RestClient.get(url)['data']
        return response

    def where(self, **kwargs) -> list[dict]:
        """Adds a parameter to the dictionary of query parameters
        
        Args:
            **kwargs: Dictionary of query parameters
        Returns:
            list of dict: List of resource dictionaries 
        """
        for key, value in kwargs.items():
            self.params[key] = value

        return self.all()
    
    def all(self) -> list[dict]:
        """Get all resources, automatically paging through data

        Returns:
            list of dict: List of resource dictionaries
        """
        data_list = []
        fetch_all = True
        url = "{BASE_URL}/{self.type.RESOURCE}"

        if 'page' in self.params:
            fetch_all = False
        else:
            self.params['page'] = 1

        while True:
            response = RestClient.get(url, self.params)['data']
            if len(response) > 0:
                data_list.extend(response)

                if fetch_all:
                    self.params['page'] += 1
                else:
                    break
            else:
                break

        return data_list


In [29]:
class Card():
    RESOURCE = 'cards'

    @staticmethod
    def find(id: str) -> dict or None:
        """Get a card by its id
        
        Args:
            id (string): Card id    
        Returns:
            dict: Dictionary containing card data
        """
        return QueryBuilder(Card).find(id)
    
    @staticmethod
    def where(**kwargs) -> list[dict]:
        """Adds a parameter to the dictionary of query parameters
        
        Args:
            **kwargs: Dictionary of query parameters
        Returns:
            list of dict: List of dictionaries containing card data
        """
        return QueryBuilder(Card).where(**kwargs)

In [30]:
class Set():
    RESOURCE = 'sets'

    @staticmethod
    def find(id: str) -> dict or None:
        """Get a set by its id
        
        Args:
            id (string): Set id    
        Returns:
            Set: Instance of the Set
        """
        return QueryBuilder(Set).find(id)
    
    @staticmethod
    def where(**kwargs) -> list[dict]:
        """Adds a parameter to the dictionary of query parameters
        
        Args:
            **kwargs: Arbitrary keyword arguments.
        Returns:
            list of dict: List of dictionaries containing set data
        """
        return QueryBuilder(Set).where(**kwargs)
    
    @staticmethod
    def all():
        """Get all sets, automatically paging through data

        Returns:
            list of dict: List of dictionaries containing set data
        """
        return QueryBuilder(Set).all()

In [32]:
class Type():
    RESOURCES = 'types'

    @staticmethod
    def all() -> list[dict]:
        """Get all types, automatically paging through data

        Returns:
            list of dict: List of dictionaries containing type data
        """
        return QueryBuilder(Type).all()
    

In [33]:
class Subtype():
    RESOURCES = 'subtypes'

    @staticmethod
    def all() -> list[dict]:
        """Get all subtypes, automatically paging through data

        Returns:
            list of dict: List of dictionaries containing subtype data
        """
        return QueryBuilder(Subtype).all()

In [ ]:
class Supertype():
    RESOURCES = 'supertypes'

    @staticmethod
    def all() -> list[dict]:
        """Get all supertypes, automatically paging through data

        Returns:
            list of dict: List of dictionaries containing supertype data
        """
        return QueryBuilder(Supertype).all()

In [ ]:
class Rarity():
    RESOURCES = 'rarities'

    @staticmethod
    def all() -> list[dict]:
        """Get all rarities, automatically paging through data

        Returns:
            list of dict: List of dictionaries containing rarity data
        """
        return QueryBuilder(Rarity).all()

In [27]:
poke_id = 'xy7-54'
url = f"{BASE_URL}/cards/{poke_id}"
response = RestClient.get(url)['data']

In [28]:
response.keys()

dict_keys(['id', 'name', 'supertype', 'subtypes', 'hp', 'types', 'evolvesFrom', 'abilities', 'attacks', 'weaknesses', 'resistances', 'retreatCost', 'convertedRetreatCost', 'set', 'number', 'artist', 'rarity', 'flavorText', 'nationalPokedexNumbers', 'legalities', 'images', 'tcgplayer', 'cardmarket'])